read data

In [1]:
import pandas as pd
import numpy as np
import os
import time
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import random
import pandas as pd

In [2]:
df=pd.read_csv('../output/tweets_with_topic_label.csv')
df=df.drop(columns=['text'])
df.head()

,id,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,Topic
0,98454970654916608,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f,0
1,1234653427789070336,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f,6
2,1218010753434820614,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f,6
3,1304875170860015617,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f,5
4,1218159531554897920,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f,9


In [3]:
df['isDeleted'].value_counts()
# imbalanced data set, consider using smote?

f    55479
t     1092
Name: isDeleted, dtype: int64

In [4]:
from sklearn import preprocessing
df=df.apply(preprocessing.LabelEncoder().fit_transform)
df.head()


,id,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,Topic
0,369,0,0,5,49,255,369,0,0
1,46117,0,0,18,14214,13112,45744,0,6
2,44514,1,0,18,0,5991,44161,0,6
3,52671,0,0,18,15275,15970,52182,0,5
4,44515,1,0,18,0,7333,44162,0,9


In [5]:
X=df.drop(columns=['isDeleted'])
Y=df['isDeleted']

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.20,random_state=0)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print('majority train class: %d' % np.sum(Y_train == 0))
print('minority train class: %d' % np.sum(Y_train == 1))
print('majority test class: %d' % np.sum(Y_test == 0))
print('minority test class: %d' % np.sum(Y_test == 1))
#imbalanced dataset

majority train class: 44389
minority train class: 867
majority test class: 11090
minority test class: 225


In [7]:
from imblearn.over_sampling import SMOTE
# using SMOTE
smt = SMOTE()

# fit and apply the transform
X_train_smote, Y_train_smote = smt.fit_resample(X_train, Y_train)
X_test_smote, Y_test_smote = smt.fit_resample(X_test, Y_test)

print('majority train class: %d' % np.sum(Y_train_smote == 0))
print('minority train class: %d' % np.sum(Y_train_smote == 1))
print('majority test class: %d' % np.sum(Y_test_smote == 0))
print('minority test class: %d' % np.sum(Y_test_smote== 1))

majority train class: 44389
minority train class: 44389
majority test class: 11090
minority test class: 11090


models

In [8]:
from sklearn.neighbors import NearestCentroid
import numpy as np
from sklearn.metrics import classification_report

clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train, Y_train)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.01 seconds
Predicting test data takes 0.004 seconds
              precision    recall  f1-score   support

           0       0.99      0.63      0.77     11090
           1       0.04      0.82      0.08       225

    accuracy                           0.63     11315
   macro avg       0.52      0.72      0.42     11315
weighted avg       0.98      0.63      0.75     11315



In [9]:
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train, Y_train)
    pre=nca_pipe.predict(X_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train, Y_train)
# Pipeline(...)
pre=nca_pipe.predict(X_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(y_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test)
print(classification_report(Y_test,pre))

In [9]:
# SGD with penalty=l1
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.367 seconds
Predicting test data takes 0.003 seconds
              precision    recall  f1-score   support

           0       0.98      0.91      0.95     11090
           1       0.07      0.29      0.11       225

    accuracy                           0.90     11315
   macro avg       0.52      0.60      0.53     11315
weighted avg       0.97      0.90      0.93     11315



In [10]:
# SGD with penalty=l2
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.354 seconds
Predicting test data takes 0.002 seconds
              precision    recall  f1-score   support

           0       1.00      0.19      0.32     11090
           1       0.02      0.96      0.05       225

    accuracy                           0.20     11315
   macro avg       0.51      0.58      0.18     11315
weighted avg       0.98      0.20      0.31     11315



In [11]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.065 seconds
Predicting test data takes 0.003 seconds
              precision    recall  f1-score   support

           0       1.00      0.82      0.90     11090
           1       0.09      0.84      0.16       225

    accuracy                           0.82     11315
   macro avg       0.54      0.83      0.53     11315
weighted avg       0.98      0.82      0.88     11315



In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 9.249 seconds
Predicting test data takes 0.321 seconds
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     11090
           1       0.91      0.40      0.56       225

    accuracy                           0.99     11315
   macro avg       0.95      0.70      0.78     11315
weighted avg       0.99      0.99      0.98     11315



In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 6.322 seconds
Predicting test data takes 6.876 seconds
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     11090
           1       0.95      0.34      0.50       225

    accuracy                           0.99     11315
   macro avg       0.97      0.67      0.75     11315
weighted avg       0.99      0.99      0.98     11315

